# Chart 1: Code consolidation

In [ ]:
import pandas as pd
import textwrap
import plotly.graph_objs as go
import plotly.io as pio
from pathlib import Path #To create unique filenames for each PFA chart
import itertools

import src.data.utilities as utils
import src.visualization.prt_theme as prt_theme

config = utils.read_config()
pio.templates.default = "prt_template"

In [ ]:
def loadData(status='interim', filename='women_cust_sentence_length_PFA_2010-2022.csv') -> pd.DataFrame:
    """Load CSV file into Pandas DataFrame and convert object columns to categories when they meet criteria in `categoryColumns()`

    Parameters
    ----------
    status : {'raw', 'interim', 'processed'}, default is 'interim'
        Status of the data processing.
        * If 'raw' file is located in "rawFilePath" within config file
        * If 'interim', file is located in "intFilePath"
        * If 'processed', file is located in "clnFilePath"
    filename : str, default is 'PFA_2010-22_women_cust_comm_sus.csv'
        Name of CSV file to be loaded.

    Returns
    -------
    DataFrame
        CSV data is returned as Pandas DataFrame with any eligible object columns converted into category columns to limit memory requirements
    """
    paths = {
        "raw": 'rawFilePath',
        "interim": 'intFilePath',
        "processed": 'clnFilePath'
    }

    dfPath=f"{config['data'][paths[status]]}{filename}"
    df = pd.read_csv(dfPath)
    print('Data loaded')
    return utils.categoryColumns(df)

In [ ]:
df = loadData()

In [ ]:
my_df = df.copy()
my_df

In [ ]:
def annotation_yvals():
    y_list = [fig.data[i]['y'][-1] for i in range(len(fig.data))] #selecting last y value for each trace 
    return y_list

In [ ]:
def trace_max():
    trace_max_list = [(fig.data[i]['y']).max() for i in range(len(fig.data))] #Selecting maximum value from each trace
    return trace_max_list

In [ ]:
def check_duplicates(y_vals, trace_max):
    duplicate_vals = [idx for idx, value in enumerate(y_vals) if y_vals.count(value) > 1]
    if len(duplicate_vals) > 0:
        print(f'Duplicates found: index {duplicate_vals}\nUpdating...')
        
        max_i = 0
        for idx in duplicate_vals:
            if trace_max[idx] > max_i:
                max_i = idx
        y_vals[max_i] = y_vals[max_i] * 1.2
        annotations[max_i]['y'] = y_vals[max_i]

In [ ]:
def check_overlap(l, space=None):
    if space is None:
        space = 0.7
    return all(x1 / x2 >= space for x1,x2 in itertools.pairwise(sorted(l)))

In [ ]:
##Refactoring
def adjust_overlap(l, space=None):
    if space is None:
        space = 0.7

    for (idx1,num1), (idx2,num2) in itertools.permutations(enumerate(l), 2):
        ratio = l[idx1] / l[idx2]
        if ratio > space:
            largest = max((idx1,num1), (idx2,num2), key=lambda x:x[1])
            largest_index = largest[0]
            l[largest_index] = l[1] / space
            # annotations[largest_index]['y'] = l[largest_index]
    return l

In [ ]:
l = [20, 27]
adjust_overlap(l)

In [ ]:
l = [20, 27]
print(l[0] / l[1])
print(l[1] / l[0])

In [ ]:
x = 20 / 0.7
x

In [ ]:
for (idx1,num1), (idx2,num2) in itertools.permutations(enumerate(l), 2):
    ratio = num1 / num2
    if ratio > 0.7:
        largest = max((idx1,num1), (idx2,num2), key=lambda x:x[1])
        smallest = min((idx1,num1), (idx2,num2), key=lambda x:x[1])
        largest_index = largest[0]
        smallest_index = smallest[0]
        l[largest_index] = l[smallest_index] / 0.7

print(largest)

In [ ]:
##Refactoring
def adjust_overlap(l, space=None):
    if space is None:
        space = 0.7

    for (idx1,num1), (idx2,num2) in itertools.permutations(enumerate(l), 2):
        ratio = num1 / num2
        if ratio > space:
            largest = max((idx1,num1), (idx2,num2), key=lambda x:x[1])
            smallest = min((idx1,num1), (idx2,num2), key=lambda x:x[1])
            largest_index = largest[0]
            smallest_index = smallest[0]
            l[largest_index] = l[smallest_index] / 0.7
    return l

In [ ]:
l = [20, 27]
adjust_overlap(l)
l

In [ ]:
df['sentence_length'].unique()

In [ ]:
df['sentence_length'] = df['sentence_length'].replace("6 months to less than 12 months", "6 months—<br>less than 12 months")

In [ ]:
for pfa in df['pfa'].unique():
    pfa_df = df[df["pfa"] == pfa]
    trace_list = [] # Need to empty my trace_list with every loop through each PFA so that charts are plotted separately
    fig = go.Figure() # Need to also instantiate the figure with every loop in order to clear fig.data values

    for i in pfa_df["sentence_length"].unique():  # Creating a for loop to extract unique values from the dataframe and make traces
        pfa_df_sentence = pfa_df[pfa_df["sentence_length"] == i]
        
        trace = go.Scatter(
            x=pfa_df_sentence["year"],
            y=pfa_df_sentence["freq"],
            mode="lines",
            name=str(pfa_df_sentence["sentence_length"].iloc[0]),
            meta=pfa_df_sentence["pfa"].iloc[0],   # Adding name of PFA in metadata to ensure data relates to only one area 
            hovertemplate="%{y}<extra></extra>"
        )

        trace_list.append(trace)

    fig.add_traces(trace_list)

    ## Chart title
    title = textwrap.wrap(f'<b>Use of immediate imprisonment for women in {pfa_df_sentence["pfa"].iloc[0]} 2010–2022</b>', width=45)

    fig.update_layout(
        margin=dict(l=63, b=75, r=100),
        title="<br>".join(title),
        yaxis_title="",
        yaxis_tickformat=",.0f",
        yaxis_tick0=0,
        xaxis_dtick=2,
        xaxis_tick0=2010,
        hovermode="x",
        width=655,
        height=500,
    )

    ## Chart annotations
    annotations = []

    # Adding trace annotations
    for j in range(0, len(trace_list)):
        annotations.append(
            dict(
                xref="x",
                yref="y",
                x=fig.data[j].x[-1],
                y=fig.data[j].y[-1],
                text=str(fig.data[j].name),
                xanchor="left",
                yanchor="bottom",
                align="left",
                showarrow=False,
                font_color=fig.layout.template.layout.colorway[j],
                font_size=10,
            )
        )

    # # Adding source label
    source = prt_theme.sourceAnnotation("Ministry of Justice, Criminal justice statistics", annotations)

    # Adding y-axis label
    annotations.append(
        dict(
            xref="x",
            yref="paper",
            x=df['year'].iloc[0],
            y=1.04,
            align="left",
            xanchor="left",
            showarrow=False,
            text="Women sentenced to custody",
            font_size=12,
        )
    )
    
    # Checking for overlapping annotations on trace labels
    annotations[1]['y'] = 0    
    
    # Adding annotations to layout
    fig.update_layout(annotations=annotations)

    ## Setting chart axis ranges
    max_y_val = 0
    for i in range(len(fig.data)):
        max_trace = (fig.data[i].y).max()
        if max_trace > max_y_val:
            max_y_val = max_trace

    y_intervals = [52, 103, 204, 305, 405, 606, 1210]
    y_max_idx = min(range(len(y_intervals)), key = lambda i: abs(y_intervals[i]-max_y_val))
    if y_intervals[y_max_idx] <= max_y_val:
        y_max = y_intervals[y_max_idx + 1]
    else: 
        y_max = y_intervals[y_max_idx]

    
    fig.update_yaxes(range=[0, y_max])
    fig.update_xaxes(range=[2009.7, 2022.3])

    # fig.show()
    # fig.show(config=config)

    ## Exporting to static image

    # Save results
    
    # export_path = f"{config['data']['outPath']}"
    export_path = Path.joinpath(Path.cwd(), f"{config['data']['outPath']}", "custody_sentence_lengths_2022")
    export_path.mkdir(parents=True, exist_ok=True) #generate if does not exist

    # Setting filename variable and full path

    filename = str(pfa_df_sentence["pfa"].iloc[0])
    export_eps_path = Path.joinpath(export_path, f'{filename}' + '.svg')

    fig.write_image(export_eps_path)

## Creating a chart class to allow for any adjustments needed (and then refactoring for final code)

In [ ]:
class SentenceLengthChart:
    
    def __init__(self, pfa, df=None, labelIDX=None, adjust=None):
        if df is None:
            df = df
        
        self.pfa = pfa
        self.df = df
        self.labelIDX = labelIDX
        self.adjust = adjust
        self.trace_list = [] # Need to empty my trace_list with every loop through each PFA so that charts are plotted separately
        self.annotations = []
        self.max_y_val = 0
        self.pfa_df_sentence = None
        self.fig = go.Figure() # Need to also instantiate the figure with every loop in order to clear fig.data values

    def break_trace_labels(self):
        self.df['sentence_length'] = self.df['sentence_length'].replace("6 months to less than 12 months", "6 months—<br>less than 12 months")

    def createTraces(self):
        pfa_df = self.df[self.df["pfa"] == self.pfa]

        for i in pfa_df["sentence_length"].unique():  # Creating a for loop to extract unique values from the dataframe and make traces
            self.pfa_df_sentence = pfa_df[pfa_df["sentence_length"] == i]
            
            trace = go.Scatter(
                x=self.pfa_df_sentence["year"],
                y=self.pfa_df_sentence["freq"],
                mode="lines",
                name=str(self.pfa_df_sentence["sentence_length"].iloc[0]),
                meta=self.pfa_df_sentence["pfa"].iloc[0],   # Adding name of PFA in metadata to ensure data relates to only one area 
                hovertemplate="%{y}<extra></extra>"
            )
            self.trace_list.append(trace)

        self.fig.add_traces(self.trace_list)

    def chartParams(self):
    ## Chart title

        title = textwrap.wrap(f'<b>Use of immediate imprisonment for women in {self.pfa_df_sentence["pfa"].iloc[0]} 2010–2022</b>', width=45)

        self.fig.update_layout(
            margin=dict(l=63, b=75, r=100),
            title="<br>".join(title),
            yaxis_title="",
            yaxis_tickformat=",.0f",
            yaxis_tick0=0,
            xaxis_dtick=2,
            xaxis_tick0=2010,
            hovermode="x",
            width=655,
            height=360,
        )

    ## Chart annotations
    def chartAnnotations(self):

        # Adding trace annotations
        for j in range(0, len(self.trace_list)):
            self.annotations.append(
                dict(
                    xref="x",
                    yref="y",
                    x=self.fig.data[j].x[-1],
                    y=self.fig.data[j].y[-1],
                    text=str(self.fig.data[j].name),
                    xanchor="left",
                    yanchor="bottom",
                    align="left",
                    showarrow=False,
                    font_color=self.fig.layout.template.layout.colorway[j],
                    font_size=10,
                )
            )
        # Adding source label
        prt_theme.source_annotation("Ministry of Justice, Criminal justice statistics", self.annotations)

        # Adding y-axis label
        self.annotations.append(
            dict(
                xref="x",
                yref="paper",
                x=df['year'].iloc[0],
                y=1.04,
                align="left",
                xanchor="left",
                showarrow=False,
                text="Women sentenced to custody",
                font_size=12,
            )
        )

        if self.labelIDX is not None and self.adjust is not None:
            self.annotations[self.labelIDX]['y'] = int(self.adjust)
        
        self.annotations[1]['y'] = 0
            
        # Adding annotations to layout
        self.fig.update_layout(annotations=self.annotations)

    def setYAxis(self):

    ## Setting chart axis ranges
    
        for i in range(len(self.fig.data)):
            max_trace = (self.fig.data[i].y).max()
            if max_trace > self.max_y_val:
                self.max_y_val = max_trace

        y_intervals = [52, 103, 210, 305, 405, 606, 1210]
        y_max_idx = min(range(len(y_intervals)), key = lambda i: abs(y_intervals[i]-self.max_y_val))
        if y_intervals[y_max_idx] <= self.max_y_val:
            y_max = y_intervals[y_max_idx + 1]
        else: 
            y_max = y_intervals[y_max_idx]

        
        self.fig.update_yaxes(range=[0, y_max])
        self.fig.update_xaxes(range=[2009.7, 2022.3])
        
    def saveChart(self, filetype='eps'):
        self.filetype = filetype

        if self.trace_list == []: #Allows saveChart method to run without outputChart requirement
            self.break_trace_labels()
            self.createTraces()
            self.chartParams()
            self.chartAnnotations()
            self.setYAxis()

        export_path = Path.joinpath(Path.cwd(), f"{config['data']['outPath']}", f"custody_sentence_lengths_2022/{self.filetype}")
        export_path.mkdir(parents=True, exist_ok=True) #generate if does not exist

        # Setting filename variable and full path
        filename = str(self.pfa_df_sentence["pfa"].iloc[0])
        export_file_path = Path.joinpath(export_path, f'{filename}.{self.filetype}')

        self.fig.write_image(export_file_path)


    def outputChart(self):
        self.break_trace_labels()
        self.createTraces()
        self.chartParams()
        self.chartAnnotations()
        self.setYAxis()
        self.fig.show()


In [ ]:
df = loadData()

In [ ]:
kent = SentenceLengthChart("Kent", df=df)
kent.outputChart()
# kent.saveChart("eps")

In [ ]:
dorset = SentenceLengthChart("Dorset", labelIDX=2, adjust=15)
dorset.outputChart()
dorset.saveChart(".svg")

In [ ]:
west_mercia = SentenceLengthChart("West Mercia", labelIDX=2, adjust=32)
west_mercia.outputChart()

In [ ]:
west_mercia.saveChart(".svg")

In [ ]:
cumbria.annotations[0]

In [ ]:
# Checking for overlapping annotations on trace labels
y_vals = annotation_yvals()
print(y_vals)

In [ ]:
adjust_overlap(y_vals)
print(y_vals)
# Adding annotations to layout
for i in range(3):
    print(annotations[i]['y'])

In [ ]:
max_trace = 0
for i in range(len(fig.data)):
    max_val = (fig.data[i].y).max()
    if max_val > max_trace:
        max_trace = max_val
    print(max_val)
max_trace

In [ ]:
y_intervals = [55, 105, 205, 305, 405, 1210]
y_max_idx = min(y_intervals, key = lambda x: abs(x-max_y_val))
if y_intervals[y_max_idx] < max_y_val:
    y_max = y_intervals[y_max_idx + 1]
else: 
    y_max = y_intervals[y_max_idx]

## Importing working script and defining manual adjustments to annotations

In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import plotly.graph_objs as go
import plotly.io as pio

import src.data.utilities as utils
from src.visualization.Chart1_Use_of_imprisonment_PFAs import SentenceLengthChart

config = utils.read_config()
pio.templates.default = "prt_template"

In [ ]:
folder = 'custody_sentence_lengths_2022'
filetype = 'pdf'

In [ ]:
df = utils.load_data(status='interim', filename='women_cust_sentence_length_PFA_2010-2022.csv')

In [ ]:
dorset = SentenceLengthChart("Dorset", df=df, label_idx=2, adjust=15)
dorset.output_chart()

In [ ]:
dorset.save_chart(folder=folder, filetype=filetype)

This is all working as it should, so let's identify those charts where changes need to be made to the placement of the annotations and store this in a dictionary.

In [ ]:
class Record:
	"""Hold a record of data."""
	def __init__(self, pfa_name, label_idx, adjust):
		self.pfa_name = pfa_name
		self.label_idx = label_idx
		self.adjust = adjust
	
	def __repr__(self) -> str:
		return f'{self.pfa_name} PFA adjustment'

In [ ]:
dorset = Record("Dorset", 2, 15)

In [ ]:
dorset.pfa_name

In [ ]:
dorset

Testing the use of these values as parameters to `SentenceLengthChart`

In [ ]:
record = dorset

In [ ]:
pfa = record.pfa_name
df = df
label_id = record.label_idx
adjust = record.adjust

In [ ]:
SentenceLengthChart(pfa, df, label_id, adjust).output_chart()

All good, now switching this up to include a list of `Record` objects

In [ ]:
pfa_adjustments = [
    Record("Dorset", 2, 15),
    Record("Cleveland", 1, 30),
    Record("Hampshire", 2, 10),
    Record("Lincolnshire", 0, 13)]

In [ ]:
for record in pfa_adjustments:
    print(record.pfa_name)

In [ ]:
for record in pfa_adjustments:
    (
    SentenceLengthChart(pfa=record.pfa_name, 
                        df=df, 
                        label_idx=record.label_idx, 
                        adjust=record.adjust
                        )
                        .output_chart())

Beautiful, this is all working as expected. Now to just build this out for all of the affected charts.

In [31]:
# OPTIONAL: Load the "autoreload" extension so that code can change
%load_ext autoreload

# OPTIONAL: always reload modules so that as you change code in {{ cookiecutter.module_name }}, it gets loaded
%autoreload 2

import pandas as pd
import plotly.graph_objs as go
import plotly.io as pio

import src.data.utilities as utils
from src.visualization.Chart1_Use_of_imprisonment_PFAs import *

config = utils.read_config()
pio.templates.default = "prt_template"


folder = 'custody_sentence_lengths_2022'
filetype = 'pdf'
df = utils.load_data(status='interim', filename='women_cust_sentence_length_PFA_2010-2022.csv')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Data loaded


In [ ]:
record = Record("Cambridgeshire", 0, 18)

In [2]:
def adjusted_chart(record: Record, output: str):
    chart = SentenceLengthChart(pfa=record.pfa_name, 
                    df=df, 
                    label_idx=record.label_idx, 
                    adjust=record.adjust
                    )
    
    if output in ['save', 'show']:
        if output == 'save':
            chart.save_chart(folder, filetype)
            return chart.output_chart()
        else:
            return chart.output_chart()
    else:
        raise ValueError("output must be 'save' or 'show'.")

In [ ]:
adjusted_chart(record=record, output='save')

In [ ]:
pfa_adjustments = [
    Record("Cambridgeshire", 0, 18),
    Record("Dorset", 2, 5),
    Record('Cumbria', 0, 7),
    Record('Derbyshire', 0, 15),
    Record('Dyfed-Powys', 0, 10),
    Record('Gloucestershire', 0, -2),
    Record('Lancashire', 2, 20),
    Record('Merseyside', 2, -10),
    Record('North Yorkshire', 0, 7),
    Record('Northumbria', 0, -10),
    Record('South Yorkshire', 0, -10),
    Record('Suffolk', 0, 8),
    Record('Sussex', 2, 20),
    Record('West Mercia', 2, 10),
    Record('West Midlands', 0, 50)]

In [13]:
record = Record('West Midlands', 0, 50)
adjusted_chart(record=record, output='show')

Label index: 0, Adjustment: 50
Applying adjustment...


In [ ]:
for record in pfa_adjustments:
    adjusted_chart(record=record, output='save')

PFAs with more than one unresolved label placement:
* Gwent
* Surrey

In [19]:
record = Record('Gwent', [0, 2], [17, 12])
adjusted_chart(record=record, output='save')

Label index: [0, 2], Adjustment: [17, 12]
Applying adjustment for multiple label indices and adjustments...
Chart saved to: /Users/alex/Documents/Coding/Python/women-pfa-2022/reports/figures/custody_sentence_lengths_2022/pdf/Gwent.pdf


In [25]:
record = Record('Surrey', [0, 1], [7, 12])
adjusted_chart(record=record, output='show')

ValueError: All values in label_idx list must be 0 or 2.

In [33]:
pfa_adjustments = [
    Record('Cambridgeshire', 0, 18),
    Record('Dorset', 2, 5),
    Record('Cumbria', 0, 7),
    Record('Derbyshire', 0, 15),
    Record('Dyfed-Powys', 0, 10),
    Record('Gloucestershire', 0, -2),
    Record('Gwent', [0, 2], [17, 12]),
    Record('Lancashire', 2, 20),
    Record('Merseyside', 2, -10),
    Record('North Yorkshire', 0, 7),
    Record('Northumbria', 0, -10),
    Record('South Yorkshire', 0, -10),
    Record('Suffolk', 0, 8),
    Record('Surrey', [0, 2], [7, 12]),
    Record('Sussex', 2, 20),
    Record('West Mercia', 2, 10),
    Record('West Midlands', 0, 50)]

In [28]:
for record in pfa_adjustments:
    adjusted_chart(record=record, output='save')

Label index: 0, Adjustment: 18
Applying adjustment...
Chart saved to: /Users/alex/Documents/Coding/Python/women-pfa-2022/reports/figures/custody_sentence_lengths_2022/pdf/Cambridgeshire.pdf


Label index: 2, Adjustment: 5
Applying adjustment...
Chart saved to: /Users/alex/Documents/Coding/Python/women-pfa-2022/reports/figures/custody_sentence_lengths_2022/pdf/Dorset.pdf


Label index: 0, Adjustment: 7
Applying adjustment...
Chart saved to: /Users/alex/Documents/Coding/Python/women-pfa-2022/reports/figures/custody_sentence_lengths_2022/pdf/Cumbria.pdf


Label index: 0, Adjustment: 15
Applying adjustment...
Chart saved to: /Users/alex/Documents/Coding/Python/women-pfa-2022/reports/figures/custody_sentence_lengths_2022/pdf/Derbyshire.pdf


Label index: 0, Adjustment: 10
Applying adjustment...
Chart saved to: /Users/alex/Documents/Coding/Python/women-pfa-2022/reports/figures/custody_sentence_lengths_2022/pdf/Dyfed-Powys.pdf


...

Label index: 0, Adjustment: 8
Applying adjustment...
Chart saved to: /Users/alex/Documents/Coding/Python/women-pfa-2022/reports/figures/custody_sentence_lengths_2022/pdf/Suffolk.pdf


Label index: [0, 2], Adjustment: [7, 12]
Applying adjustment for multiple label indices and adjustments...
Chart saved to: /Users/alex/Documents/Coding/Python/women-pfa-2022/reports/figures/custody_sentence_lengths_2022/pdf/Surrey.pdf


Label index: 2, Adjustment: 20
Applying adjustment...
Chart saved to: /Users/alex/Documents/Coding/Python/women-pfa-2022/reports/figures/custody_sentence_lengths_2022/pdf/Sussex.pdf


Label index: 2, Adjustment: 10
Applying adjustment...
Chart saved to: /Users/alex/Documents/Coding/Python/women-pfa-2022/reports/figures/custody_sentence_lengths_2022/pdf/West Mercia.pdf


Label index: 0, Adjustment: 50
Applying adjustment...
Chart saved to: /Users/alex/Documents/Coding/Python/women-pfa-2022/reports/figures/custody_sentence_lengths_2022/pdf/West Midlands.pdf


In [30]:
for record in pfa_adjustments:
    print(record.pfa_name)

Cambridgeshire
Dorset
Cumbria
Derbyshire
Dyfed-Powys
Gloucestershire
Gwent
Lancashire
Merseyside
North Yorkshire
Northumbria
South Yorkshire
Suffolk
Surrey
Sussex
West Mercia
West Midlands


In [38]:
filename = 'women_cust_sentence_length_PFA_2010-2022.csv'
folder = 'custody_sentence_lengths_2022'
pfa_adjustments = [
    Record('Cambridgeshire', 0, 18),
    Record('Dorset', 2, 5),
    Record('Cumbria', 0, 7),
    Record('Derbyshire', 0, 15),
    Record('Dyfed-Powys', 0, 10),
    Record('Gloucestershire', 0, -2),
    Record('Gwent', [0, 2], [17, 12]),
    Record('Lancashire', 2, 20),
    Record('Merseyside', 2, -10),
    Record('North Yorkshire', 0, 7),
    Record('Northumbria', 0, -10),
    Record('South Yorkshire', 0, -10),
    Record('Suffolk', 0, 8),
    Record('Surrey', [0, 2], [7, 12]),
    Record('Sussex', 2, 20),
    Record('West Mercia', 2, 10),
    Record('West Midlands', 0, 50)]

make_pfa_sentence_length_charts(filename, folder, output='show', pfa_adjustments=pfa_adjustments)

Data loaded
Label index: 0, Adjustment: 0


Label index: 0, Adjustment: 0


Label index: 0, Adjustment: 18
Applying adjustment...


Label index: 0, Adjustment: 0


Label index: 0, Adjustment: 0


...

Label index: 2, Adjustment: 10
Applying adjustment...


Label index: 0, Adjustment: 50
Applying adjustment...


Label index: 0, Adjustment: 0


Label index: 0, Adjustment: 0


Charts ready


Nice snippet below to use `setattr` to store values from a dictionary

In [ ]:
class Record:
	"""Hold a record of data."""

In [ ]:
john = {
    "name": "John Doe",
    "position": "Python Developer",
    "department": "Engineering",
    "salary": 80000,
    "hire_date": "2020-01-01",
    "is_manager": False,
}

In [ ]:
john_record = Record()

In [ ]:
for field, value in john.items():
    setattr(john_record, field, value)

In [ ]:
john_record.position